In [5]:
import torch
from transformers import AutoModel, AutoTokenizer
import yaml
import requests
from underthesea import text_normalize

In [6]:
torch.cuda.is_available()

True

In [7]:
from sentence_transformers import SentenceTransformer, util

sent_model = SentenceTransformer('keepitreal/vietnamese-sbert')

In [4]:
import py_vncorenlp
word_segmenter = py_vncorenlp.VnCoreNLP(annotators=['wseg'], save_dir="/workspace/nlplab/quannd/scada-full-stack/chatbot-scada/vncorenlp")

2024-02-23 19:58:45 INFO  WordSegmenter:24 - Loading Word Segmentation model


In [6]:
# import torch.nn.functional as F
# device = "cuda" if torch.cuda.is_available() else "cpu"
# tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
# model = AutoModel.from_pretrained("vinai/phobert-base-v2").to(device)

# bos_token_id = tokenizer.bos_token_id
# eos_token_id = tokenizer.eos_token_id


In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [9]:
raw_nlu_file = "/workspace/nlplab/quannd/scada-full-stack/raw_nlu/raw_nlu.yml"
f = open(raw_nlu_file, "r")
raw_nlu_data = yaml.load(f, Loader=yaml.FullLoader)
f.close()

In [10]:
for intent in raw_nlu_data.keys():
    print(intent, ": ", raw_nlu_data[intent]) 

affirm :  ['ừ', 'đúng rồi', 'đúng là như vậy', 'chuẩn rùi', 'chính xác']
bot_challenge :  ['cu có phải bot real ko ?', 'cậu là sinh viên trường nào thế ?', 'Hãy giới thiệu bản thân cho tớ nghe đi.', 'huh ?', 'Cậu là bot hay là người thế ?', 'thế cậu là người hay là bot vậy ?', 'cậu là bot hay là người thế ?']
change_bottom :  ['Hãy di chuyển hình chữ nhật màu tím có kích thước 150x100 pixel, độ dày đường viền là 4 pixel, đang ở vị trí (250, 350) trên bản vẽ, đến vị trí (450, 550) theo tọa độ góc bên dưới của nó.', 'Tôi muốn bạn di chuyển hình bầu dục màu cam có chiều dài trục lớn là 120 pixel, chiều dài trục nhỏ là 80 pixel, đang ở bên phải bản vẽ, đến vị trí có tọa độ (300, 400) theo góc bên dưới của hình này.', 'Hãy di chuyển hình tròn màu tím đến vị trí (400, 500) theo góc bên dưới của nó.', 'Tôi muốn bạn di chuyển hình ellipse đang ở bên phải bản vẽ đến vị trí có tọa độ (200, 300) theo góc bên dưới của nó.', 'Bạn có thể di chuyển hình tam giác màu xanh dương đến vị trí (500, 600) t

In [11]:
len(raw_nlu_data['vertical_flip'])

54

In [31]:
# raw_nlu_file = "/workspace/nlplab/quannd/scada-full-stack/raw_data_by_intent/raw_nlu.yml"
# f = open(raw_nlu_file, "r")
# raw_nlu_data = yaml.load(f, Loader=yaml.FullLoader)
# f.close()
# for intent in raw_nlu_data.keys():
#     raw_intent_sent = raw_nlu_data[intent]
#     raw_intent_sent_id_list = []
#     for i in range(len(raw_intent_sent)):
#         line = raw_intent_sent[i]
#         payload = {"text": line}
#         response = requests.post(url="http://192.168.16.5:9091/vncorenlp", json=payload)
#         words = response.json()['tokens']
#         words = [word.replace(" ", "_") for word in words]
#         sent = " ".join(words)
#         raw_intent_sent_id_list.append(tokenizer.encode(sent))
    
#     raw_intent_sent_vector_list = []
    
#     with torch.no_grad():
#         for id in raw_intent_sent_id_list:
#             inputs = torch.tensor([id])
#             inputs = inputs.to(device)
#             features = model(inputs)

#             pooler_output = features[1]
#             if device == "cuda":
#                 pooler_output = pooler_output.cpu().detach().numpy()
#             raw_intent_sent_vector_list.append(pooler_output)
#         raw_intent_sent_vector_np = np.concatenate(raw_intent_sent_vector_list, axis=0)
#         np.save(f"/workspace/nlplab/quannd/scada-full-stack/raw_nlu_embedding_by_intent/{intent}.npy", raw_intent_sent_vector_np)


In [32]:
# def cosine_sim_check(intent: str):

#     # raw_intent_sent_vector_list = np.load(f"/workspace/nlplab/quannd/scada-full-stack/raw_nlu_embedding_by_intent/{intent}.npy", mmap_mode="r")
#     raw_intent_sent = raw_nlu_data[intent]
#     raw_intent_sent_id_list = []
#     for i in range(len(raw_intent_sent)):
#         line = raw_intent_sent[i]
#         payload = {"text": line}
#         response = requests.post(url="http://192.168.16.5:9091/vncorenlp", json=payload)
#         words = response.json()['tokens']
#         words = [word.replace(" ", "_") for word in words]
#         sent = " ".join(words)
#         raw_intent_sent_id_list.append(tokenizer.encode(sent))
    
#     raw_intent_sent_vector_list = []
    
#     with torch.no_grad():
#         for id in raw_intent_sent_id_list:
#             inputs = torch.tensor([id])
#             inputs = inputs.to(device)
#             features = model(inputs)

#             pooler_output = features[1]
#             if device == "cuda":
#                 pooler_output = pooler_output.cpu().detach().numpy()
#             raw_intent_sent_vector_list.append(pooler_output)
#         raw_intent_sent_vector_np = np.concatenate(raw_intent_sent_vector_list, axis=0)
#         print(raw_intent_sent_vector_np)


#     sent_id_list = []
#     f = open(f"/workspace/nlplab/quannd/scada-full-stack/raw_data_by_intent/{intent}.txt", "r")
#     for line in f.readlines():
#         payload = {"text": line.rstrip("\n ")}
#         response = requests.post(url="http://192.168.16.5:9091/vncorenlp", json=payload)
#         words = response.json()['tokens']
#         words = [word.replace(" ", "_") for word in words]
#         sent = " ".join(words)
#         sent_id_list.append(tokenizer.encode(sent))
#     f.close()

    
#     sent_vector_list = []

#     with torch.no_grad():
#         for id in sent_id_list:
#             inputs = torch.tensor([id])
#             inputs = inputs.to(device)
#             features = model(inputs)

#             pooler_output = features[1]
#             if device == "cuda":
#                 pooler_output = pooler_output.cpu().detach().numpy()
#             sent_vector_list.append(pooler_output)
        
#         sent_vector_np = np.concatenate(sent_vector_list, axis=0)
#         cos = cosine_similarity(sent_vector_np, raw_intent_sent_vector_np)

#         print(cos)
#         good_sentences_check = [False] *  len(cos)
#         for i, sim_list in enumerate(cos):
#             if  np.mean(sim_list) <= 0.9 and np.mean(sim_list) >= 0.75 and max(sim_list) < 1:
#                 good_sentences_check[i] = True
    
#         result_sent = []
#         f = open(f"/workspace/nlplab/quannd/scada-full-stack/raw_data_by_intent/{intent}.txt", "r")
#         for line in f.readlines():
#             result_sent.append(line)
#         f.close()

#         result = []
#         for sent, check in zip(result_sent, good_sentences_check):
#             result.append((sent, check))
#         return cos, result

In [14]:
def cosine_sim_check_with_sentence_bert(intent: str):

    raw_intent_sent = raw_nlu_data[intent]
    for i, line in enumerate(raw_intent_sent):

        line = line.rstrip("\n ")
        tmp_sent = word_segmenter.word_segment(text_normalize(line))
        sent = " ".join(tmp_sent)

        # payload = {"text": line.rstrip("\n ")}
        # response = requests.post(url="http://192.168.16.5:9091/vncorenlp", json=payload)
        # words = response.json()['tokens']
        # words = [word.replace(" ", "_") for word in words]
        # sent = " ".join(words)

        raw_intent_sent[i] = sent

    raw_intent_sent_id_np = sent_model.encode(raw_intent_sent)


    sent_list = []
    initial_sent_list = []
    f = open(f"/workspace/nlplab/quannd/scada-full-stack/raw_data_by_intent/{intent}.txt", "r")
    for line in f.readlines():
        initial_sent_list.append(line)

        line = line.rstrip("\n ")
        tmp_sent = word_segmenter.word_segment(text_normalize(line))
        sent = " ".join(tmp_sent)

        # payload = {"text": line.rstrip("\n ")}
        # response = requests.post(url="http://192.168.16.5:9091/vncorenlp", json=payload)
        # words = response.json()['tokens']
        # words = [word.replace(" ", "_") for word in words]
        # sent = " ".join(words)
        
        sent_list.append(sent)
    f.close()

    sent_vector_np = sent_model.encode(sent_list)
    print(sent_vector_np.shape)
    cos = util.cos_sim(sent_vector_np, raw_intent_sent_id_np)

    print(cos.shape)
    cos = cos.numpy().tolist()
    good_sentences_check = [False] *  len(cos)
    for i, sim_list in enumerate(cos):
        if  np.mean(sim_list) <= 0.7 and np.mean(sim_list) >= 0.3 and max(sim_list) < 1:
            good_sentences_check[i] = True
    
    result = []
    for sent, check in zip(initial_sent_list, good_sentences_check):
        result.append((sent, check))
    return cos, result

In [15]:
import os
dir_path = "/workspace/nlplab/quannd/scada-full-stack/raw_data_by_intent"
intent_list = os.listdir(dir_path)

In [16]:
intent_list = [intent[:-4] for intent in intent_list]
# intent_list = ["delete_selected_objects", "select_all_object", "exit_select", "select_area"]
print(intent_list)

['delete_selected_objects', 'change_top', 'move_right', 'change_bottom', 'rotate_right', 'cut_selected_objects', 'redo', 'change_right', 'color_background', 'move_up', 'color_foreground', 'move_left', 'select_area', 'copy_selected_objects', 'draw_parallelogram', 'undo', 'exit_select', 'paste', 'select_all_object']


In [17]:
dir_path

'/workspace/nlplab/quannd/scada-full-stack/raw_data_by_intent'

In [19]:
length_limit = 65

In [20]:
for intent in intent_list:
    len_intent = len(raw_nlu_data[intent])
    print(intent)
    if len_intent < length_limit:
        cos, tmp_list = cosine_sim_check_with_sentence_bert(intent)
        nesessary_length = length_limit - len_intent
        nesessary_sent_list = []
        tmp_true_sent_list = [sent for sent, check in tmp_list if check == True]
        if len(tmp_true_sent_list) >= nesessary_length:
            nesessary_sent_list = tmp_true_sent_list[0: nesessary_length]
            print(len(nesessary_sent_list))
            print("good enough!")
            nesessary_string = "\n".join(nesessary_sent_list)
            with open(f"{dir_path}/{intent}.txt", "w") as f_open:
                f_open.write(nesessary_string)
            f_open.close()
        else:
            print("not enough!")

delete_selected_objects
(12, 768)
torch.Size([12, 44])
not enough!
change_top
(12, 768)
torch.Size([12, 54])
11
good enough!
move_right
(43, 768)
torch.Size([43, 54])
11
good enough!
change_bottom
(37, 768)
torch.Size([37, 54])
11
good enough!
rotate_right
(12, 768)
torch.Size([12, 54])
11
good enough!
cut_selected_objects
(30, 768)
torch.Size([30, 54])
not enough!
redo
(5, 768)
torch.Size([5, 49])
not enough!
change_right
(21, 768)
torch.Size([21, 54])
11
good enough!
color_background
(72, 768)
torch.Size([72, 53])
12
good enough!
move_up
(42, 768)
torch.Size([42, 54])
11
good enough!
color_foreground
(12, 768)
torch.Size([12, 55])
10
good enough!
move_left
(42, 768)
torch.Size([42, 54])
11
good enough!
select_area
(12, 768)
torch.Size([12, 40])
not enough!
copy_selected_objects
(30, 768)
torch.Size([30, 49])
not enough!
draw_parallelogram
undo
(10, 768)
torch.Size([10, 48])
not enough!
exit_select
(12, 768)
torch.Size([12, 50])
not enough!
paste
(18, 768)
torch.Size([18, 45])
not eno